<a href="https://colab.research.google.com/github/OJShearer/EPL-21-22-xG/blob/main/Understat_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Aim is to get data for *ExG* from [Understat](https://understat.com) for EPL season 21/22 so far, and each team's statistics.

In [3]:
url = 'https://understat.com/league/EPL/2021'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')

# Inspecting the website source code, data is JSON under <script> tags
scripts = soup.find_all('script')

In [ ]:
# Check our <script> tags
for i in scripts:
  print('*'*50)
  print(i.text)

# Data we want is in variables teamsData and playersData 

In [5]:
import json

string_with_json_obj = ''

# Find teams data
for i in scripts:
  if 'teamsData' in str(i):
    string_with_json_obj = str(i).strip()

# print(string_with_json_obj)

# Get just the JSON data part by index slicing
start = string_with_json_obj.index("(") + 2
end = string_with_json_obj.index("')")
json_data = string_with_json_obj[start:end]

json_data = json_data.encode('utf8').decode('unicode_escape')

In [6]:
# Explore how the data looks
data = json.loads(json_data)
print(data.keys())
print('-'*50)
print(data['71'].keys())
print('-'*50)
print(data['71']['id'])
print('-'*50)
print(data['71']['title'])
print('-'*50)
print(data['71']['history'][0])

dict_keys(['71', '72', '74', '75', '78', '79', '80', '81', '82', '83', '86', '87', '88', '89', '90', '92', '220', '229', '244', '245'])
--------------------------------------------------
dict_keys(['id', 'title', 'history'])
--------------------------------------------------
71
--------------------------------------------------
Aston Villa
--------------------------------------------------
{'h_a': 'a', 'xG': 1.13718, 'xGA': 1.35036, 'npxG': 0.376011, 'npxGA': 1.35036, 'ppda': {'att': 182, 'def': 28}, 'ppda_allowed': {'att': 303, 'def': 21}, 'deep': 6, 'deep_allowed': 4, 'scored': 2, 'missed': 3, 'xpts': 1.1959, 'result': 'l', 'date': '2021-08-14 14:00:00', 'wins': 0, 'draws': 0, 'loses': 1, 'pts': 0, 'npxGD': -0.974349}


Data is a dictionary of dictionaries, with 3 keys: *id, title, and history* - id also as the first layer key. History has a list of dictionaries with the stats for each league match played.

In [7]:
# Get team names and ids into a dictionary
teams = {}
for id in data.keys():
  teams[id] = data[id]['title']

In [8]:
# Get columns from history

columns = list(data['71']['history'][0].keys())
print(columns)

['h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins', 'draws', 'loses', 'pts', 'npxGD']


In [9]:
# Getting data for one
AV_data = []
for i in data['71']['history']:
  AV_data.append(list(i.values()))

df = pd.DataFrame(AV_data, columns=columns)
df.head()

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,a,1.137180,1.350360,0.376011,1.350360,"{'att': 182, 'def': 28}","{'att': 303, 'def': 21}",6,4,2,3,1.1959,l,2021-08-14 14:00:00,0,0,1,0,-0.974349
1,h,1.184260,0.631630,0.423091,0.631630,"{'att': 202, 'def': 14}","{'att': 150, 'def': 17}",5,4,2,0,1.9441,w,2021-08-21 14:00:00,1,0,0,3,-0.208539
2,h,0.431464,1.133120,0.431464,1.133120,"{'att': 264, 'def': 20}","{'att': 199, 'def': 24}",2,8,1,1,0.6785,d,2021-08-28 14:00:00,0,1,0,1,-0.701656
3,a,1.173490,1.221530,1.173490,1.221530,"{'att': 306, 'def': 31}","{'att': 188, 'def': 19}",6,3,0,3,1.3221,l,2021-09-11 16:30:00,0,0,1,0,-0.048040
4,h,0.777716,0.755035,0.777716,0.755035,"{'att': 148, 'def': 18}","{'att': 202, 'def': 17}",5,2,3,0,1.3358,w,2021-09-18 16:30:00,1,0,0,3,0.022681


In [10]:
# Get data for all teams
dataframes = {}
for id, team in teams.items():
  teams_data = []
  for row in data[id]['history']:
    teams_data.append(list(row.values()))
    
  df = pd.DataFrame(teams_data, columns=columns)
  dataframes[team] = df
  print('Added data for {}.'.format(team))

Added data for Aston Villa.
Added data for Everton.
Added data for Southampton.
Added data for Leicester.
Added data for Crystal Palace.
Added data for Norwich.
Added data for Chelsea.
Added data for West Ham.
Added data for Tottenham.
Added data for Arsenal.
Added data for Newcastle United.
Added data for Liverpool.
Added data for Manchester City.
Added data for Manchester United.
Added data for Watford.
Added data for Burnley.
Added data for Brighton.
Added data for Wolverhampton Wanderers.
Added data for Brentford.
Added data for Leeds.


Have a dictionary of dataframes with key as team name and value the dataframe of statistics.

In [11]:
dataframes['Liverpool'].head()

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,a,1.78728,1.333300,1.78728,1.333300,"{'att': 359, 'def': 31}","{'att': 308, 'def': 12}",13,3,3,0,1.7064,w,2021-08-14 16:30:00,1,0,0,3,0.453980
1,h,2.63945,0.690464,2.63945,0.690464,"{'att': 91, 'def': 13}","{'att': 280, 'def': 22}",8,3,2,0,2.5683,w,2021-08-21 11:30:00,1,0,0,3,1.948986
2,h,2.08462,0.573638,1.32345,0.573638,"{'att': 211, 'def': 29}","{'att': 309, 'def': 11}",6,4,1,1,2.4875,d,2021-08-28 16:30:00,0,1,0,1,0.749812
3,a,4.40160,0.652514,4.40160,0.652514,"{'att': 252, 'def': 15}","{'att': 304, 'def': 8}",15,8,3,0,2.9307,w,2021-09-12 15:30:00,1,0,0,3,3.749086
4,h,2.76793,0.644168,2.76793,0.644168,"{'att': 155, 'def': 30}","{'att': 244, 'def': 12}",8,8,3,0,2.7229,w,2021-09-18 14:00:00,1,0,0,3,2.123762


In [12]:
# Get coefficient for ppda and oppda
for team, df in dataframes.items():
  dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
  dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)

dataframes['Liverpool'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,...,xpts,result,date,wins,draws,loses,pts,npxGD,ppda_coef,oppda_coef
0,a,1.78728,1.333300,1.78728,1.333300,"{'att': 359, 'def': 31}","{'att': 308, 'def': 12}",13,3,3,...,1.7064,w,2021-08-14 16:30:00,1,0,0,3,0.453980,11.580645,25.666667
1,h,2.63945,0.690464,2.63945,0.690464,"{'att': 91, 'def': 13}","{'att': 280, 'def': 22}",8,3,2,...,2.5683,w,2021-08-21 11:30:00,1,0,0,3,1.948986,7.000000,12.727273


In [20]:
# drop unwanted columns
for team, df in dataframes.items():
  dataframes[team].drop(['ppda','ppda_allowed','date'], axis=1, inplace=True)

dataframes['Liverpool'].head(2)

,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,missed,xpts,result,wins,draws,loses,pts,npxGD,ppda_coef,oppda_coef
0,a,1.78728,1.333300,1.78728,1.333300,13,3,3,0,1.7064,w,1,0,0,3,0.453980,11.580645,25.666667
1,h,2.63945,0.690464,2.63945,0.690464,8,3,2,0,2.5683,w,1,0,0,3,1.948986,7.000000,12.727273


In [21]:
# Create concatenated dataframe with all match data and hierarchical index
match_data = pd.concat(list(dataframes.values()), keys=list(dataframes.keys()))

In [24]:
# Create aggregated table
cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
cols_to_mean = ['ppda_coef', 'oppda_coef']

frames = []
for team, df in dataframes.items():
  # Transpose taken as methods return series
  sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
  mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
  final_df = sum_data.join(mean_data)
  final_df['team'] = team
  final_df['matches'] = len(df)
  frames.append(final_df)
  
table_data = pd.concat(frames)

In [25]:
# Reorder columns, sort by points
table_data = table_data[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
table_data.sort_values('pts', ascending=False, inplace=True)
table_data.reset_index(inplace=True, drop=True)
table_data['position'] = range(1,len(table_data)+1)

In [26]:
# Create xG differences
table_data['xG_diff'] = table_data['xG'] - table_data['scored']
table_data['xGA_diff'] = table_data['xGA'] - table_data['missed']
table_data['xpts_diff'] = table_data['xpts'] - table_data['pts']

In [27]:
# Set integers
cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
table_data[cols_to_int] = table_data[cols_to_int].astype(int)

In [30]:
col_order = ['position','team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
table_data = table_data[col_order]
pd.options.display.float_format = '{:,.2f}'.format
table_data.head(10)

,position,team,matches,wins,draws,loses,scored,missed,pts,xG,...,xGA,xGA_diff,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts,xpts_diff
0,1,Manchester City,29,22,4,3,68,18,70,69.29,...,19.61,1.61,18.85,44.89,9.09,28.25,375,112,68.89,-1.11
1,2,Liverpool,29,21,6,2,75,20,69,75.60,...,26.31,6.31,26.31,43.86,8.49,24.62,346,127,66.72,-2.28
2,3,Chelsea,28,17,8,3,57,19,59,49.48,...,25.32,6.32,21.51,22.65,9.95,17.50,270,125,53.63,-5.37
3,4,Arsenal,28,17,3,8,44,31,54,45.30,...,35.38,4.38,33.09,8.29,15.07,14.20,231,166,47.54,-6.46
4,5,Tottenham,29,16,3,10,47,36,51,49.98,...,37.00,1.00,35.47,12.23,14.47,14.05,216,200,48.12,-2.88
5,6,Manchester United,29,14,8,7,48,40,50,47.56,...,42.49,2.49,39.44,5.83,14.39,13.02,223,180,44.64,-5.36
6,7,West Ham,30,14,6,10,49,39,48,45.07,...,41.56,2.56,39.28,1.83,14.32,14.37,194,205,44.35,-3.65
7,8,Wolverhampton Wanderers,30,14,4,12,31,26,46,31.78,...,41.01,15.01,37.21,-6.18,17.36,11.69,139,238,35.72,-10.28
8,9,Aston Villa,29,11,3,15,41,40,36,31.52,...,40.50,0.50,36.58,-6.58,13.91,9.18,200,190,34.17,-1.83
9,10,Leicester,27,10,6,11,42,46,36,35.92,...,49.16,3.16,43.73,-9.34,16.25,12.60,181,228,30.32,-5.68


In [31]:
# Data to csv
match_data.to_csv('match.csv')
table_data.to_csv('table.csv')